## Explore PubAG API

In [8]:
import requests
import json

In [2]:
API_KEY = "jGgEv0rYItqngfgOd9eF9HN5GoCDRxTzB64slFdL"

In [53]:
response = requests.get("https://api.nal.usda.gov/pubag/rest/search/?query=precision+agriculture&per_page=100&page=22&api_key={}".format(API_KEY)).json()

In [47]:
response

{'version': 1.0,
 'hitCount': 2939,
 'request': {'query': 'precision agriculture',
  'page': 8,
  'totalPages': 30,
  'pageSize': 100},
 'resultList': [{'id': '6521360',
   'agid': ['6521360'],
   'title': 'A Review of Methods to Improve Nitrogen Use Efficiency in Agriculture',
   'timestamp': '2021-02-03T02:56:03Z',
   'author': ['Sharma, Lakesh K.', 'Bali, Sukhwinder K.'],
   'author_primary': 'Sharma, Lakesh K.',
   'subject': ['application methods',
    'corn',
    'fertilizers',
    'leaf area index',
    'nitrogen',
    'normalized difference vegetation index',
    'nutrient use efficiency',
    'plant tissues',
    'planting',
    'potatoes',
    'precision agriculture',
    'remote sensing',
    'soil',
    'soil analysis'],
   'source': '2017 v.10 no.1',
   'journal': 'Sustainability',
   'abstract': 'Management of nitrogen (N) is a challenging task and several methods individually and in combination are in use to manage its efficiency. However, nitrogen use efficiency (NUE) h

In [34]:
totalpages = response["request"]["totalPages"]
results = response["resultList"]
pageSize

abstracts = []
for i in range(totalpages):
    abstracts.append(results[i]["abstract"]) 

In [57]:
abstract_dict_file = "abs_dict_pubag.json"

with open(abstract_dict_file) as f:
    abs_dict = json.load(f)

# abs_dict = {}

In [58]:
len(abs_dict["titles"])

76944